In [10]:
import torch
from PIL import Image
import open_clip
import ipyplot
import matplotlib.pyplot as plt

from matplotlib.pyplot import figure, imshow, axis
from matplotlib.image import imread

torch.set_printoptions(precision=3, sci_mode=False)

ModuleNotFoundError: No module named 'open_clip'

In [ ]:
def showImagesHorizontally(list_of_files):
    fig = figure()
    number_of_files = len(list_of_files)
    for i in range(number_of_files):
        a=fig.add_subplot(1,number_of_files,i+1)
        image = imread(list_of_files[i])
        # imshow(image, cmap='Greys_r')
        imshow(image)
        axis('off')

# Load Model

In [4]:
open_clip.list_models()

NameError: name 'open_clip' is not defined

In [ ]:
model_b, _, preprocess_b = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
tokenizer_b = open_clip.get_tokenizer('ViT-B-32')

In [5]:
model_l, _, preprocess_l = open_clip.create_model_and_transforms('ViT-L-14-336', pretrained='openai')
tokenizer_l = open_clip.get_tokenizer('ViT-B-32')

NameError: name 'open_clip' is not defined

In [6]:
text_b = tokenizer_b(["a diagram", "a lobster", "a cat"])
text_l = tokenizer_l(["a diagram", "a lobster", "a cat"])

NameError: name 'tokenizer_b' is not defined

# Show Images

<div style="display: flex; flex-direction: row;">
    <img src="./experiment_data/lobster_grayscale_0.png" style="width: 200px; height: auto; margin-right: 10px;">
    <img src="./experiment_data/lobster_blue_0.png" style="width: 200px; height: auto; margin-right: 10px;">
    <img src="./experiment_data/lobster_red_0.png" style="width: 200px; height: auto; margin-right: 10px;">
</div>

In [ ]:
lobster_0_files = [
    "./experiment_data/lobster_grayscale_0.png",
    "./experiment_data/lobster_blue_0.png",
    "./experiment_data/lobster_red_0.png"
]

lobster_1_files = [
    "./experiment_data/lobster_grayscale_1.png",
    "./experiment_data/lobster_blue_1.png",
    "./experiment_data/lobster_red_1.png"
]

lobster_2_files = [
    "./experiment_data/lobster_grayscale_2.png",
    "./experiment_data/lobster_blue_2.png",
    "./experiment_data/lobster_red_2.png"
]

lobster_real_files = [
    "./experiment_data/rlobster_0.png",
    "./experiment_data/rlobster_1.png",
    "./experiment_data/rlobster_2.png"
    "./experiment_data/rlobster_3.png"
    "./experiment_data/rlobster_4.png"
]

# showImagesHorizontally(lobster_0_files)

# Lobster (Top)

In [ ]:
ipyplot.plot_images(lobster_0_files, max_images=3, img_width=250)

# Lobster (Front)

In [7]:
ipyplot.plot_images(lobster_1_files, max_images=3, img_width=250)

NameError: name 'ipyplot' is not defined

# Lobster (Bottom)

In [ ]:
ipyplot.plot_images(lobster_2_files, max_images=3, img_width=250)

# Lobster Images

In [ ]:
ipyplot.plot_images(lobster_real_files, max_images=5, img_width=250)

In [ ]:
for filepath in lobster_0_files:
    image = preprocess_b(Image.open(filepath)).unsqueeze(0)
    with torch.no_grad(), torch.cuda.amp.autocast():
        image_features = model_b.encode_image(image)
        text_features = model_b.encode_text(text_b)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        
        dist = image_features @ text_features.T
        cos_sim = dist / (torch.norm(image_features, dim=-1) * torch.norm(text_features, dim=-1))
        text_probs = (100.0 * dist).softmax(dim=-1)
    
    print("Image: ", filepath.split("/")[-1])
    print(f"Label probs: {text_probs}")
    print(f"Cosine similarity: {cos_sim}")
    print()

In [ ]:
for filepath in lobster_1_files:
    image = preprocess_b(Image.open(filepath)).unsqueeze(0)
    with torch.no_grad(), torch.cuda.amp.autocast():
        image_features = model_b.encode_image(image)
        text_features = model_b.encode_text(text_b)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        
        dist = image_features @ text_features.T
        cos_sim = dist / (torch.norm(image_features, dim=-1) * torch.norm(text_features, dim=-1))
        text_probs = (100.0 * dist).softmax(dim=-1)
    
    print("Image: ", filepath.split("/")[-1])
    print(f"Label probs: {text_probs}")
    print(f"Cosine similarity: {cos_sim}")
    print()

In [ ]:
for filepath in lobster_2_files:
    image = preprocess_b(Image.open(filepath)).unsqueeze(0)
    with torch.no_grad(), torch.cuda.amp.autocast():
        image_features = model_b.encode_image(image)
        text_features = model_b.encode_text(text_b)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        
        dist = image_features @ text_features.T
        cos_sim = dist / (torch.norm(image_features, dim=-1) * torch.norm(text_features, dim=-1))
        text_probs = (100.0 * dist).softmax(dim=-1)
    
    print("Image: ", filepath.split("/")[-1])
    print(f"Label probs: {text_probs}")
    print(f"Cosine similarity: {cos_sim}")
    print()

# CLIP: ViT-L-14 (Larger model)

In [8]:
for filepath in lobster_0_files:
    image = preprocess_l(Image.open(filepath)).unsqueeze(0)
    with torch.no_grad(), torch.cuda.amp.autocast():
        image_features = model_l.encode_image(image)
        text_features = model_l.encode_text(text_b)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        
        dist = image_features @ text_features.T
        cos_sim = dist / (torch.norm(image_features, dim=-1) * torch.norm(text_features, dim=-1))
        text_probs = (100.0 * dist).softmax(dim=-1)
    
    print("Image: ", filepath.split("/")[-1])
    print(f"Label probs: {text_probs}")
    print(f"Cosine similarity: {cos_sim}")
    print()

NameError: name 'lobster_0_files' is not defined

In [9]:
for filepath in lobster_1_files:
    image = preprocess_l(Image.open(filepath)).unsqueeze(0)
    with torch.no_grad(), torch.cuda.amp.autocast():
        image_features = model_l.encode_image(image)
        text_features = model_l.encode_text(text_b)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        
        dist = image_features @ text_features.T
        cos_sim = dist / (torch.norm(image_features, dim=-1) * torch.norm(text_features, dim=-1))
        text_probs = (100.0 * dist).softmax(dim=-1)
    
    print("Image: ", filepath.split("/")[-1])
    print(f"Label probs: {text_probs}")
    print(f"Cosine similarity: {cos_sim}")
    print()

NameError: name 'lobster_1_files' is not defined

In [ ]:
for filepath in lobster_2_files:
    image = preprocess_l(Image.open(filepath)).unsqueeze(0)
    with torch.no_grad(), torch.cuda.amp.autocast():
        image_features = model_l.encode_image(image)
        text_features = model_l.encode_text(text_b)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        
        dist = image_features @ text_features.T
        cos_sim = dist / (torch.norm(image_features, dim=-1) * torch.norm(text_features, dim=-1))
        text_probs = (100.0 * dist).softmax(dim=-1)
    
    print("Image: ", filepath.split("/")[-1])
    print(f"Label probs: {text_probs}")
    print(f"Cosine similarity: {cos_sim}")
    print()

In [ ]:
for filepath in lobster_real_files:
    image = preprocess_l(Image.open(filepath)).unsqueeze(0)
    with torch.no_grad(), torch.cuda.amp.autocast():
        image_features = model_l.encode_image(image)
        text_features = model_l.encode_text(text_b)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        
        dist = image_features @ text_features.T
        cos_sim = dist / (torch.norm(image_features, dim=-1) * torch.norm(text_features, dim=-1))
        text_probs = (100.0 * dist).softmax(dim=-1)
    
    print("Image: ", filepath.split("/")[-1])
    print(f"Label probs: {text_probs}")
    print(f"Cosine similarity: {cos_sim}")
    print()